In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'dataset/training_set/'
validation_data_dir = 'dataset/test_set/'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [3]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [5]:
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [6]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/50
125/125 [==============================] - 88s 703ms/step - loss: 0.6899 - acc: 0.5320 - val_loss: 0.6870 - val_acc: 0.5587
Epoch 2/50
125/125 [==============================] - 83s 661ms/step - loss: 0.6914 - acc: 0.5375 - val_loss: 0.6863 - val_acc: 0.6100
Epoch 3/50
125/125 [==============================] - 77s 617ms/step - loss: 0.6890 - acc: 0.5420 - val_loss: 0.6694 - val_acc: 0.6088
Epoch 4/50
125/125 [==============================] - 86s 692ms/step - loss: 0.6846 - acc: 0.5490 - val_loss: 0.6720 - val_acc: 0.5863
Epoch 5/50
125/125 [==============================] - 89s 708ms/step - loss: 0.6737 - acc: 0.5995 - val_loss: 0.6470 - val_acc: 0.6375
Epoch 6/50
125/125 [==============================] - 84s 674ms/step - loss: 0.6392 - acc: 0.6485 - val_loss: 0.6166 - val_acc: 0.6300
Epoch 7/50
125/125 [==============================] - 83s 662ms/step - loss: 0.6412 - acc: 0.6445 - val_lo

In [55]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/test_set/cats/cat.4532.jpg', target_size=(150, 150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
# result = model.predict(test_image)
preds = model.predict_classes(test_image)
probs = model.predict_proba(test_image)
print(preds, probs)


[[0]] [[ 0.]]


In [56]:
if preds[0][0] > 0.5:
    pred = 'dog'
else:
    pred = 'cat'
print(pred)

cat


In [57]:
model.save_weights('first_try.h5')